# Notebook to Sub file Tex

## Objective

This script is part of the hybrid chapter workflow, and to be used when you have created a ipython notebook and want to convert it to sub file tex where its preamble is removed, and instead added from the style file from our hybrid template. This sub file can be compiled separately and also becomes part of chapter, when included in the main tex file. 

## Rules
1. Try to name the notebook without any spaces (use underscore instead). 
2. Do not manually number the equations. Instead use align

## Remember
1. This only creates tex. You need to include it accordingly in main file. By default main tex has documentclass book, so recommended to add as a chapter.  

## Input Parameters

In [33]:
source_file = "../chapters/ipy_sample.ipynb"  # file to be converted
tex_output_folder = "../chapters"
template_file = "ipy2tex_subfile_template.tplx"
temp_folder = "../tmp"
nbconvert_config_file = "nbconvert_config.py"

import ntpath
tex_file = ntpath.basename(source_file).split('.ipynb')[0] + ".tex"
tex_file_with_path_wo_ext = tex_output_folder + "/" + ntpath.basename(source_file).split('.ipynb')[0]

## Steps

1. Create the template file. This is optional once the file is created. It is here just to show whats going on in template mostly. 
2. Generate the tex file using the template file and ```nbconvert```.

> A sample notebook is used to show the demo here

### Create Template

In [34]:
%%writefile $template_file

((*- extends 'article.tplx' -*))

% 1. CHANGE DOCUMENTCLASS TO STANDALONE
((* block docclass *))
\documentclass[float=false,crop=false]{standalone}
((* endblock docclass *))



% 2. REMOVE PREAMBLE AND REPLACE WITH STYLE FILE INCLUSION
((* block packages *))
\usepackage{../myipy2tex}  % NOTE WE ARE ASSSUMING THE STYLE FILE TO BE ONE FOLDER ABOVE

% if you need to cross reference to any raw tex file from this resultant tex file you  need to refer them here..
% it is not needed when you compile main.tex but make sure the labels are unique
\IfEq{\jobname}{\detokenize{main}}{}{%
    \externaldocument{raw_sample} 
} 


((* endblock packages *))
((* block definitions *))
((* endblock definitions *))
((* block commands *))
((* endblock commands *))

% Draw a figure using the graphicx package. Min size due to tikzmagic outputs being too shrinked.
((* macro draw_figure(filename) -*))
((* set filename = filename | posix_path *))
((*- block figure scoped -*))
    \begin{center}
    \adjustimage{max size={0.9\linewidth}{0.9\paperheight},min size={0.5\linewidth}{!}}{((( filename )))}
    \end{center}
    { \hspace*{\fill} \\}
((*- endblock figure -*))
((*- endmacro *))


% 3. BELOW STUFF ARE JUST FOR CODE CELL CORRECTIONS TO GET LINES WRAPPED PROPERLY..
%===============================================================================
% Input
%===============================================================================

((* block input scoped *))
    ((*- if 'to_remove' in cell.metadata.get('tags', []) -*))  % remove input part of cells with tag to_remove
    %((- if cell.metadata.hide_input -))
    ((*- else -*))
        ((( custom_add_prompt(cell.source | wrap_text(88) | highlight_code(strip_verbatim=True), cell, 'In', 'incolor') )))    
    ((*- endif -*))
((* endblock input *))

%===============================================================================
% Output
%===============================================================================

% Display stream ouput with coloring
((* block stream *))
    \begin{Verbatim}[commandchars=\\\{\},fontsize=\footnotesize]
((( output.text | wrap_text(86) | escape_latex | ansi2latex )))
    \end{Verbatim}
((* endblock stream *))

%==============================================================================
% Define macro custom_add_prompt() (derived from add_prompt() macro in style_ipython.tplx)
%==============================================================================

((* macro custom_add_prompt(text, cell, prompt, prompt_color) -*))
    ((*- if cell.execution_count is defined -*))
    ((*- set execution_count = "" ~ (cell.execution_count | replace(None, " ")) -*))
    ((*- else -*))
    ((*- set execution_count = " " -*))
    ((*- endif -*))
    ((*- set indention =  " " * (execution_count | length + 7) -*))
\begin{((( prompt ~ 'Verbatim')))}[commandchars=\\\{\},fontsize=\scriptsize]
((( text | add_prompts(first='{\color{' ~ prompt_color ~ '}' ~ prompt ~ '[{\\color{' ~ prompt_color ~ '}' ~ execution_count ~ '}]:} ', cont=indention) )))
\end{((( prompt ~ 'Verbatim')))}
((*- endmacro *))

Overwriting ipy2tex_subfile_template.tplx


### Create Tex

By default, the attachments in markdown cells are not exported, so we will have a small config file to force that as well

In [35]:
%%writefile $nbconvert_config_file

c.NbConvertApp.export_format = 'latex'
c.LatexExporter.preprocessors = [
    'extract_attachments.ExtractAttachmentsPreprocessor'  # custom preprocessor from extract_attachments.py
    ]

Overwriting nbconvert_config.py


In [ ]:
!jupyter nbconvert --log-level='DEBUG' --template "$template_file" "$source_file" --output-dir="$tex_output_folder" --config "$nbconvert_config_file"